# Cellpose demo

This notebook is an interactive sandbox for testing Cellpose on your data to determine which channels yield the best segmentation results.

Transient Testing: To allow for rapid iteration, fragments are processed and added to the SpatialData object in-memory only.

No Disk Output: These tests are not saved to disk, ensuring you can experiment with different parameters without modifying your original files.

Note that Cellpose doesn't provide matching cells and nuclei masks. To get both masks the segmenter needs to be run twice (on appropriate input) and matching of the objects needs to be done downstream (not supported by PlexPipe).

See also: https://cellpose.readthedocs.io/en/latest/index.html.

In [1]:
from pathlib import Path
import numpy as np
from cellpose import models
import spatialdata as sd
from napari_spatialdata import Interactive
from spatialdata.transformations import Translation, set_transformation
from spatialdata.models import Labels2DModel

d:\plex-pipe\.venv\Lib\site-packages\dask\dataframe\__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(
d:\plex-pipe\.venv\Lib\site-packages\xarray_schema\__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution


In [6]:
model = models.CellposeModel(gpu=True)

## Read in data

In [2]:
sdata_path = Path.cwd().parents[1] / "examples/output/sample_analysis/rois/ROI_000.zarr"
sdata = sd.read_zarr(sdata_path)
sdata

SpatialData object, with associated Zarr store: D:\plex-pipe\examples\output\sample_analysis\rois\ROI_000.zarr
├── Images
│     ├── 'CD45': DataTree[cyx] (1, 4800, 4864), (1, 2400, 2432), (1, 1200, 1216)
│     ├── 'DAPI': DataTree[cyx] (1, 4800, 4864), (1, 2400, 2432), (1, 1200, 1216)
│     └── 'NaKATPase': DataTree[cyx] (1, 4800, 4864), (1, 2400, 2432), (1, 1200, 1216)
├── Labels
│     ├── 'cytoplasm': DataTree[yx] (4800, 4864), (2400, 2432), (1200, 1216)
│     ├── 'instanseg_cell': DataTree[yx] (4800, 4864), (2400, 2432), (1200, 1216)
│     ├── 'instanseg_nucleus': DataTree[yx] (4800, 4864), (2400, 2432), (1200, 1216)
│     └── 'ring': DataTree[yx] (4800, 4864), (2400, 2432), (1200, 1216)
├── Shapes
│     ├── 'qc_exclude_CD45': GeoDataFrame shape: (1, 1) (2D shapes)
│     └── 'qc_exclude_DAPI': GeoDataFrame shape: (2, 1) (2D shapes)
└── Tables
      └── 'instanseg_table': AnnData (12451, 25)
with coordinate systems:
    ▸ 'global', with elements:
        CD45 (Images), DAPI (Images),

## Parameters

In [3]:
# Choose position of a fragment to test
row_start = 2000
column_start = 2000
height = 1000
width = 1000

# create input image for segmentation
channels = ['DAPI']

## Try segmentation with selected channels

In [4]:
# create input sample for segmentation
input_image = [np.array(sdata[ch]['scale0'].image[:,row_start:row_start+height, column_start:column_start+width]) for ch in channels]
input_image = np.stack(input_image).squeeze()
input_image.shape

(1000, 1000)

In [7]:
# run segmentation
masks, _, _ = model.eval(input_image)

## Visualize segmentation in napari

In [8]:
# add segmentation sample to the SpatialData object
transform = Translation([row_start, column_start], axes=('y', 'x'))

nucleus_element = Labels2DModel.parse(
    masks, 
    dims=('y', 'x'), 
    transformations={"global": transform}
)

sdata.labels["nuclei"] = nucleus_element

In [9]:
interactive = Interactive(sdata)
interactive.run()

2026-02-12 14:09:46.537 | WARNING  | napari_spatialdata._viewer:__init__:57 - Due to Shift-L being used as shortcut in napari, it is being deprecated and might not link a new layer to an existing SpatialData object in the viewer. Please use ⌘-L on MacOS or else Ctrl-L.


D:\uv\python\cpython-3.12-windows-x86_64-none\Lib\functools.py:912: ImplicitModificationWarning: Transforming to str index.
  return dispatch(args[0].__class__)(*args, **kw)
2026-02-12 14:10:01.696 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2026-02-12 14:10:01.700 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2026-02-12 14:10:06.225 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
2026-02-12 14:10:06.240 | DEBUG    | napari_spatialdata._view:_on_layer_update:569 - Updating layer.
